## Whole brain calcium imaging data from C. elegans, Kato et al. 2015

Kato, S., Kaplan, H. S., Schrödel, T., Skora, S., Lindsay, T. H., Yemini, E., et al. (2015). Global Brain Dynamics Embed the Motor Command Sequence of Caenorhabditis elegans. Cell, 163(3), 656–669. http://doi.org/10.1016/j.cell.2015.09.034

In [2]:
!ls
from datetime import datetime
from dateutil.tz import tzlocal

import matplotlib.pyplot as plt
%matplotlib inline 
import pynwb
import math
from hdmf.backends.hdf5.h5_utils import H5DataIO

import h5py
import numpy as np
import scipy.io as sio



all_datarefs = ['WT_Stim']

for data_ref in all_datarefs:
    
    mat_file = '%s.mat'%data_ref
    h5_file = h5py.File(mat_file, 'r')
    mat_contents = h5_file[data_ref]


    print('Contents of MAT file %s: %s'%(mat_file, sorted(mat_contents.keys())))
    
    '''
    From readme:
    
    traces_raw= neural activity traces uncorrected
    traces = neural activity traces corrected for bleaching
    tracesDif = derivative of traces
    IDs = identified neuron IDs
    timeVectorSeconds = time vector in seconds
    fps = frames per second
    dataset = name of dataset
    stimulus
        -identity = what was changed e.g. O2 (oxygen)
        -type = stimulus type e.g. binary steps
        -switchtimes =  time in seconds when stimulus changed from initial state to the other state
        -initialstate = the state that the stimulus starts with, refers to "conc"
        -conc = the concentrations of the stimulus
        -concunits - units of the "conc"

    States = vector of different state types (8 states for WT_NoStim, 4 states for WT_Stim and AVA_HisCl

    8 states for WT_NoStim:
        ‘FWD’ forward crawling; ‘SLOW’ forward slowing; ‘DT’ dorsal post reversal turn
        ‘VT’ ventral post reversal turn; ‘REV1’ reverse crawling;
        ‘REV2’ reverse crawling; ‘REVSUS’ sustained reverse crawling; ‘NOSTATE’ - ambiguous

    4 states for WT_Stim and AVA_HisCl:
        ‘FWD’ forward crawling; ‘REV’ reverse crawling
        ‘REVSUS’ sustained reverse crawling; ‘TURN’ post reversal turn
    '''
    
    datasets = [0,1]
    datasets = range(7)
    for dataset_index in datasets:
        
        start_time = datetime.now(tz=tzlocal())
        create_date = datetime.now(tz=tzlocal())

        experimenter = '??? (Zimmer lab)'


        print('\n========  loading dataset id %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['dataset'])
        raw = h5_file[raw_h5ref[dataset_index][0]]
        #print(raw)
        st = [r[0] for r in raw]
        dataset_id = ''.join(map(chr,st))
        print(dataset_id)
        dataset_id 
        
        main_ref = 'Kato et al. 2015 dataset: %s taken from file %s.mat'%(dataset_id,data_ref)
        nwbfile = pynwb.NWBFile(main_ref, 
                      main_ref, 
                      start_time,
                      file_create_date=create_date,
                      notes='NWB file created with pynwb v%s'%pynwb.__version__,
                      experimenter=experimenter,
                      experiment_description='ED...',
                      institution='IN...')
        
        print('\n========  loading fps %i'%dataset_index)
        fps_raw_h5ref = np.array(mat_contents['fps'])
        fps_raw = h5_file[fps_raw_h5ref[dataset_index][0]]
        #print(fps_raw)
        print(fps_raw[0][0])
        
        
        print('\n========  loading IDs %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['IDs'])
        print('len: %s'%len(raw_h5ref[dataset_index]))
        raw = h5_file[raw_h5ref[dataset_index][0]]
        print(raw)
        ID_info = {}
        for ri in range(len(raw)):
            r = raw[ri]
            #print('ID %i: %s, %i'%(ri, r, len(r)))
            ref = h5_file[r[0]]
            #print('  > %s (%s)'%(ref[0], type(ref[0])))
            if ref[0] != 0:
                ii = h5_file[ref[0][0]]
                #print('    > %s: %s'%(ii, ii.value))
                if len(ii.shape)==2:
                    st = [r[0] for r in ii]
                    s2 = ''.join(map(chr,st))
                    #print(s2)
                    ID_info[ri] = '%sOrMore'%s2  # TODO: fix for case more than one cell assigned to this recording location!!
                else:
                    ID_info[ri] = 'UnknownB'
            else:
                ID_info[ri] = 'UnknownA'
        print(ID_info)

        print('\n========  loading timeVectorSeconds %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['timeVectorSeconds'])
        raw = h5_file[raw_h5ref[dataset_index][0]]
        print('Data points %s: %s'%(raw.shape, raw[0]))
        timestamps = raw[0]
        
        trace_types = {'traces_raw':'Neural activity traces uncorrected',
                       'traces': 'Neural activity traces corrected for bleaching',
                       'tracesDif': 'Derivative of traces'}
        
        for trace_type in trace_types:
            print('\n========  loading %s %i'%(trace_type,dataset_index))

            raw_h5ref = np.array(mat_contents[trace_type])
            raw = h5_file[raw_h5ref[dataset_index][0]]
            #print(raw)
            #print(raw.value)
            #print(raw.value[0])

            for i in ID_info:
                data = raw.value[i]
                id = ID_info[i]
                comments='Extracted from MAT file: %s.mat'%data_ref
                wrapped_data = H5DataIO(data=data, compression=True) 
                ref = '%s__%s__%s'%(trace_type, i, id)
                desc = '%s; ID: %s; cell: %s'%(trace_types[trace_type],i,id)
                #print('Adding: %s'%desc)
                ts_acq = pynwb.TimeSeries(ref, wrapped_data, 'none', timestamps=timestamps,comments=comments,
                                         description=desc)
                nwbfile.add_acquisition(ts_acq)
        
        
        print('\n========  loading States %i'%dataset_index)

        raw_h5ref = np.array(mat_contents['States'])
        raw = h5_file[raw_h5ref[dataset_index][0]]
        states = np.array([s[0] for s in raw])
        print('Data points %s: %s'%(len(states), states))
        
        data = states

        comments='Extracted from MAT file: %s.mat'%data_ref
        wrapped_data = H5DataIO(data=data, compression=True) 
        
        if data_ref=='WT_NoStim':
            #8 states for WT_NoStim:
            state_info = "‘FWD’ forward crawling; ‘SLOW’ forward slowing; ‘DT’ dorsal post reversal turn"+\
                         "; ‘VT’ ventral post reversal turn; ‘REV1’ reverse crawling;"+\
                         "; ‘REV2’ reverse crawling; ‘REVSUS’ sustained reverse crawling; ‘NOSTATE’ - ambiguous"

        if data_ref=='WT_Stim' or data_ref=='AVA_HisCl':
            #4 states for WT_Stim and AVA_HisCl:
            state_info = "‘FWD’ forward crawling; ‘REV’ reverse crawling"+\
                         "; ‘REVSUS’ sustained reverse crawling; ‘TURN’ post reversal turn"

        ts_acq = pynwb.TimeSeries('States', wrapped_data, 'state', timestamps=timestamps,comments=comments,
                                 description='Vector of different state types: %s'%state_info)
        nwbfile.add_acquisition(ts_acq)


        nwb_file_name = 'KatoEtAl2018.%s.%i.nwb'%(data_ref, dataset_index)
        io = pynwb.NWBHDF5IO(nwb_file_name, mode='w')
        io.write(nwbfile)
        io.close()
        print("Written NWB file to %s"%nwb_file_name)
    
    '''
    plt.figure()
    id = '???'
    for i in range(15,25,1):
        id = h5_file[IDs[i][0]].value[0]
        print('ID: %s (%s)'%(id, type(id))) 
        if not type(id)==np.uint64:
            id = [int(a) for a in h5_file[id[0]].value]
            id = ''.join(chr(i) for i in id)
        else:
            id = '???'
        print('Plotting cell %i: %s'%(i,id))
        plt.plot(timeVectorSeconds.value[0],traces_raw.value[i], lw=.5, label='%i: %s'%(i,id))
        plt.legend()

    plt.show()    '''


AVA_HisCl.mat              KatoEtAl2018.WT_Stim.6.nwb
KatoEtAl2018.WT_Stim.0.nwb TestData.ipynb
KatoEtAl2018.WT_Stim.1.nwb WT_NoStim.mat
KatoEtAl2018.WT_Stim.2.nwb WT_NoStim_pre73.mat
KatoEtAl2018.WT_Stim.3.nwb WT_Stim.mat
KatoEtAl2018.WT_Stim.4.nwb readme_Kato2015.txt
KatoEtAl2018.WT_Stim.5.nwb
Contents of MAT file WT_Stim.mat: ['IDs', 'States', 'dataset', 'fps', 'stimulus', 'timeVectorSeconds', 'traces', 'tracesDif', 'traces_raw']

========  loading dataset id 0
TS20140624b_lite-1_punc-31_NLS3_4eggs_56um_1mMTet_basalplusstim_720s

========  loading fps 0
3.0527777777777776

========  loading IDs 0
len: 1
<HDF5 dataset "w": shape (107, 1), type "|O">
{0: 'UnknownA', 1: 'UnknownA', 2: 'UnknownA', 3: 'UnknownA', 4: 'UnknownA', 5: 'UnknownA', 6: 'UnknownA', 7: 'UnknownA', 8: 'UnknownA', 9: 'UnknownA', 10: 'UnknownA', 11: 'UnknownA', 12: 'UnknownA', 13: 'UnknownA', 14: 'UnknownA', 15: 'UnknownA', 16: 'IL1DROrMore', 17: 'UnknownA', 18: 'UnknownA', 19: 'UnknownA', 20: 'UnknownA', 21: 'IL1DL


========  loading traces 3

========  loading tracesDif 3

========  loading States 3
Data points 2018: [2. 2. 2. ... 1. 1. 1.]
Written NWB file to KatoEtAl2018.WT_Stim.3.nwb

========  loading dataset id 4
TS20140725b_lite-1_punc-31_NLS3_AVB_AIY_AIA_4eggs_56um_1mMTet_basalplus6stim_720s

========  loading fps 4
3.0569444444444445

========  loading IDs 4
len: 1
<HDF5 dataset "7g": shape (123, 1), type "|O">
{0: 'UnknownA', 1: 'UnknownA', 2: 'UnknownA', 3: 'UnknownA', 4: 'UnknownA', 5: 'UnknownA', 6: 'UnknownA', 7: 'UnknownA', 8: 'UnknownA', 9: 'UnknownA', 10: 'UnknownA', 11: 'UnknownA', 12: 'UnknownA', 13: 'UnknownA', 14: 'UnknownA', 15: 'UnknownA', 16: 'UnknownA', 17: 'URYDROrMore', 18: 'URYVLOrMore', 19: 'URYVROrMore', 20: 'URADLOrMore', 21: 'URADROrMore', 22: 'URYDLOrMore', 23: 'UnknownA', 24: 'UnknownA', 25: 'UnknownA', 26: 'BAGLOrMore', 27: 'OLQVROrMore', 28: 'UnknownA', 29: 'UnknownA', 30: 'OLQDLOrMore', 31: 'OLQVLOrMore', 32: 'OLQDROrMore', 33: 'BAGROrMore', 34: 'RMEROrMore', 


========  loading traces 6

========  loading tracesDif 6

========  loading States 6
Data points 2019: [3. 3. 3. ... 4. 4. 4.]
Written NWB file to KatoEtAl2018.WT_Stim.6.nwb
